## First Look at Mag Data

#### Import Data Files

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import hvplot.pandas
import numpy as np
import matplotlib.dates as dates
import warnings
warnings.filterwarnings('ignore')
import holoviews as hv
from holoviews import dim, opts
import hvplot.dask
hv.extension('bokeh')
import glob, os
import dask.dataframe as dd
import datetime as dt

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

### Start a Dask Cluster

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
def parse_date(date_string):
    year   = date_string[3:5]
    month  = date_string[0:2]
    day    = date_string[7:9]
    hour   = date_string[10:12]
    minute = date_string[13:15]
    second = date_string[16:18]
#     if day == '032':
#         print(date_string)
#         month = '02'
#         day = '01'
    
    common_date_string = year   + '-' + \
                         month  + '-' + \
                         day    + ' ' + \
                         hour   + ':' + \
                         minute + ':' + \
                         second
    
    dt = datetime.datetime.strptime(common_date_string, 
                                    "%y-%m-%d %H:%M:%S")
    
    return dt

In [ ]:
#JDAY parser
def parse_date(date_string):
    year   = date_string[0:2]
    jday  = date_string[3:6]
    hour   = date_string[10:12]
    minute = date_string[13:15]
    second = date_string[16:18]
#     if day == '032':
#         print(date_string)
#         month = '02'
#         day = '01'
    
    common_date_string = year   + '-' + \
                         month  + '-' + \
                         day    + ' ' + \
                         hour   + ':' + \
                         minute + ':' + \
                         second
    
    dt = datetime.datetime.strptime(common_date_string, 
                                    "%y-%m-%d %H:%M:%S")
    
    return dt

In [ ]:
#!ls /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/february/

In [ ]:
#!ls /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/january/

In [ ]:
cat /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/february/*.mag > /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt

In [ ]:
cat /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/january/*.mag >> /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt

In [ ]:
!head /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt

In [ ]:
#!head /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/february/temp.txt

In [ ]:
!rm /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt
!rm /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/magnetometro.csv
!cat /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/february/*.mag \
> /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt
!cat /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/january/*.mag \
>> /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt
!sed -ibak -e 's/*19./2019,/g' -e 's/\//,/' -e 's/.5 F:/,/g' -e 's/.7 F:/,/g' -e 's/ S:/,/g' \
-e 's/ D:/,/g' -e 's/m A:/,/g' -e 's/m L0 /,/' -e 's/ Q:/,/' -e 's/     X:/,/' \
-e 's/ Y:/,/' -e 's/ Z:/,/' -e 's/ x:/,/' -e 's/ y:/,/' -e 's/ <INTERP>//' -e 's/ <REAL>//' \
-e 's/ <LAYBACK INTERPOLATION>//' -e '/2019,0/!d' /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt
#!sed '/019,0/!d' /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt
!echo 'year,JD,time,f_mag,S:,depth(m), dummy,A (m):,LO,Xgrid,Ygrid,zone,longitude,latitude' \
| cat - /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/bigfile.txt \
> /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/magnetometro.csv
!head /Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/magnetometro.csv

In [ ]:
from dask import delayed
import dask.array as dsa

In [ ]:
%%time
df_magnet=pd.read_csv('/Users/dsoule/data/bravoseis_data/MAGNETOMETRO/dotmagFiles/magnetometro.csv')
df_magnet['date'] = pd.to_datetime(df_magnet.year, format='%Y') + pd.to_timedelta(df_magnet.JD - 1, unit='d')
df_magnet['time'] = pd.to_datetime(df_magnet['time'],format= '%H:%M:%S').dt.time
df_magnet['Timestamp'] = pd.to_datetime(df_magnet["date"].astype(str) + " " + df_magnet["time"].astype(str))
df_magnet=df_magnet.set_index("Timestamp")
df_magnet.head(5)

In [ ]:
df_magnet['Timestamp'] = df_magnet.apply(lambda r : pd.datetime.combine(r['date_column_name'],r['time_column_name']).time(),1)
